# IMPORTS E CONFIGURAÇÕES

  ## Imports

In [ ]:
import sys
import datetime
import pandas as pd
import numpy as np
import pickle as pkl

## Configurações

In [ ]:
# plt.rcParams['figure.figsize'] = (20, 16)
pd.set_option('display.max_columns', None)

# CARREGAMENTO DOS DADOS

## Carregar

In [ ]:
# LOAD DATA
df_sales = pd.read_csv('../data/raw/train.csv', low_memory=False)
df_store = pd.read_csv('../data/raw/store.csv', low_memory=False)

## Mesclar

In [ ]:
# MERGE DATA
df_raw = pd.merge(
    df_sales, 
    df_store, 
    on='Store', 
    how='left'
)

## Filtrar

In [ ]:
# Considera apenas as lojas que estão abertas e que possuem vendas.
df1 = (
    df_raw[
        (df_raw['Open'] == 1) & 
        (df_raw['Sales'] > 0)
    ].copy()
)

# PREPARAÇÃO DOS DADOS

## Entendimento dos Dados

### Descrição das Features

| Proprieade | Descrição |
| ---: | :-- |
| **Id** | um Id que representa um duple (Loja, Data) dentro do conjunto de teste |
| **Store** | um ID exclusivo para cada loja |
| **Sales** | o faturamento de um determinado dia (é isso que você está prevendo) |
| **Customers** | o número de clientes em um determinado dia |
| **Open** | um indicador para saber se a loja estava aberta: <br><br>0 = fechada, <br>1 = aberta |
| **StateHoliday** | indica feriado estadual. Normalmente todas as lojas, com poucas exceções, <br>fecham nos feriados estaduais. Observe que todas as escolas estão fechadas nos <br>feriados e fins de semana. <br><br>a = feriado público, <br>b = feriado de Páscoa, <br>c = Natal, <br>0 = Nenhum |
| **SchoolHoliday** | indica se a (Loja, Data) foi afetada pelo fechamento de escolas públicas |
| **StoreType** | 4 modelos de loja diferentes: <br><br>a, b, c, d |
| **Assortment** | descreve um nível de sortimento: <br><br>a = básico, <br>b = extra, <br>c = estendido |
| **CompetitionDistance** | distância em metros até a loja concorrente mais próxima |
| **CompetitionOpenSince[Month/Year]** | fornece o ano e mês aproximados da hora em que o concorrente mais próximo foi aberto |
| **Promo** | indica se uma loja está realizando uma promoção naquele dia |
| **Promo2** | Promo2 é uma promoção contínua e consecutiva para algumas lojas: <br><br>0 = loja não participa, <br>1 = loja participa |
| **Promo2Since[Year/Week]** | descreve o ano e a semana do calendário em que a loja começou a participar da Promo2 |
| **PromoInterval** | descreve os intervalos consecutivos de início da Promo2, nomeando os meses em que a <br>promoção é reiniciada. Por exemplo, "fevereiro, maio, agosto, novembro" significa que cada <br>rodada começa em fevereiro, maio, agosto e novembro de qualquer ano para aquela loja |

## Descrição dos Dados

### Visualizar

In [ ]:
df_raw.head()

### Tamanho

In [ ]:
df_raw.shape

### Tipos e Nomes das Colunas

In [ ]:
df_raw.dtypes

### Verificar Valores Nulos

In [ ]:
df_raw.isnull().sum()

### Verificar Valores Duplicados

In [ ]:
df_raw.duplicated().sum()

## Limpeza dos Dados

### Tratamento de NAs

**Premissas**
| FEATURES                       | TOTAL DE NAs | JUSTIFICATIVA |
|--------------------------------|--------------| --------------|
| CompetitionDistance            | 2.642        | Caso o valor seja NA, será atribuído um número bem elevado, pois a ausência de informação pode indicar que não há concorrentes próximos ou a distância é extremamente grande.|
| CompetitionOpenSinceMonth/Year | 323.348      | Se estiver faltando o valor, será utilizado o mês/ano correspondente da coluna 'date'. |
| Promo2SinceWeek/Year           | 508.031      | Na ausência de informação, será considerado a semana/ano presentes na coluna 'date'. |
| PromoInterval                  | 508.031      | Se não houver valor, será preenchido com none. |

In [ ]:
df1['Date'] = pd.to_datetime(df1.Date, format='%Y-%m-%d')
df1['CompetitionDistance'] = df1['CompetitionDistance'].fillna(200000.0)
df1['CompetitionOpenSinceYear'] = df1['CompetitionOpenSinceYear'].fillna(df1['Date'].dt.year)
df1['CompetitionOpenSinceMonth'] = df1['CompetitionOpenSinceMonth'].fillna(df1['Date'].dt.month)
df1['Promo2SinceWeek'] = df1['Promo2SinceWeek'].fillna(df1['Date'].dt.isocalendar().week)
df1['Promo2SinceYear'] = df1['Promo2SinceYear'].fillna(df1['Date'].dt.year)
df1['PromoInterval'] = df1['PromoInterval'].fillna('none')
df1.isnull().sum()

### Alterar Tipos

In [ ]:
df1['CompetitionOpenSinceMonth'] = df1['CompetitionOpenSinceMonth'].astype(int)
df1['CompetitionOpenSinceYear'] = df1['CompetitionOpenSinceYear'].astype(int)
df1['Promo2SinceWeek'] = df1['Promo2SinceWeek'].astype(int)
df1['Promo2SinceYear'] = df1['Promo2SinceYear'].astype(int)
df1['CompetitionDistance'] = df1['CompetitionDistance'].astype(int)

# Valores categóricos
df1['PromoInterval'] = df1['PromoInterval'].astype('category')
df1['StateHoliday'] = df1['StateHoliday'].astype('category')
df1['StoreType'] = df1['StoreType'].astype('category')
df1['Assortment'] = df1['Assortment'].astype('category')

# Valores booleanos
df1['Promo'] = df1['Promo'].astype('bool')
df1['Promo2'] = df1['Promo2'].astype('bool')
df1['SchoolHoliday'] = df1['SchoolHoliday'].astype('bool')

df1.dtypes

### Criar Novas Features

In [ ]:
# Split Date Features
df1['Year'] = df1.Date.dt.year.astype(int)
df1['Month'] = df1.Date.dt.month.astype(int)
df1['Day'] = df1.Date.dt.day.astype(int)
df1['MonthYear'] = df1.Date.dt.strftime('%Y-%m')
df1['YearWeek'] = df1.Date.dt.strftime( '%Y-%W' )
df1['WeekOfYear'] = df1.Date.dt.isocalendar().week.astype(int)
df1['DayOfYear'] = df1.Date.dt.day_of_year.astype(int)

# Criação de coluna identificando se o período é promocional (1) ou não (0)
MonthMap = {
    1: 'Jan', 
    2: 'Fev', 
    3: 'Mar', 
    4: 'Apr', 
    5: 'May', 
    6: 'Jun', 
    7: 'Jul', 
    8: 'Aug', 
    9: 'Sep', 
    10: 'Oct', 
    11: 'Nov',
    12: 'Dec'
}

df1['MonthMap'] = df1['Date'].dt.month.map(MonthMap)

df1['IsPromo'] = (
    df1[['PromoInterval', 'MonthMap']].apply(
        lambda x: 
            0 if x['PromoInterval'] == 0 
        else 
            1 if x['MonthMap'] in x['PromoInterval'].split(',') 
        else 0, 
        axis = 1)
)

# Criação de coluna identificando o tempo em meses que há competidores ativos referente àquela loja
df1['CompetitionSince'] = df1.apply(
    lambda x: datetime.datetime(
        year = x['CompetitionOpenSinceYear'], 
        month = x['CompetitionOpenSinceMonth'], 
        day = 1), 
    axis = 1
)
df1['CompetitionTimeMonth'] = ((df1['Date'] - df1['CompetitionSince'])/30).apply(lambda x: x.days).astype(int)
# CompetitionSince: coluna que junta a data em ano, mês e dia
# day = 1: só tem o ano e o mês, considerando o mês como um todo (como se todo mês estivesse no mesmo dia)
# datetime.datetime: um é um método e o outro é uma classe
# diminui a data pela data de competição para saber o período de duração da competição
# dividir por 30 para manter a granularidade mínima como mês

# Criação de coluna identificando o tempo em semanas que há competidores ativos referente àquela loja
df1['PromoSince'] = df1['Promo2SinceYear'].astype(str) + '-' + df1['Promo2SinceWeek'].astype(str) # ano-semana
df1['PromoSince'] = df1['PromoSince'].apply(lambda x: datetime.datetime.strptime(x + '-1', '%Y-%W-%w') - datetime.timedelta(days = 7)) # ano-mes-dia
df1['PromoTimeWeek'] = ((df1['Date'] - df1['PromoSince'])/7).apply(lambda x: x.days).astype(int) # quantidade de semanas
# timedelta(days = 7): para converter uma semana numa diferença de 7 dias
# x + -1: pega sempre o dia anterior em relação a contagem das semanas (exemplo: 2013-01 = 2012-12-31)

In [ ]:
# Visualização das novas features
df1[[
    'Year',
    'Month',
    'Day',
    'MonthYear',
    'YearWeek',
    'WeekOfYear',
    'DayOfYear',
    'MonthMap',
    'IsPromo',
    'CompetitionSince',
    'CompetitionTimeMonth',
    'PromoSince',
    'PromoTimeWeek'
]].sample(5)

### Alterar Valores

In [ ]:
# StateHoliday
# a = feriado público
# b = feriado de Páscoa
# c = Natal
# 0 = Nenhum
df1['StateHoliday'] = df1['StateHoliday'].cat.rename_categories({
    'a':'public', 
    'b':'easter',
    'c':'christmas',
    '0':'none'})

# Assortment
# a = básico
# b = extra
# c = estendido
df1['Assortment'] = df1['Assortment'].cat.rename_categories({
    'a':'basic',
    'b':'extra',
    'c':'extended'
})
df1['Assortment'] = pd.Categorical(
    df1['Assortment'],
    categories=[
        'basic', 
        'extra', 
        'extended'
    ],
    ordered=True
)

df1[[
    'StateHoliday',
    'Assortment',
]].sample(5)

### Remover Features

In [ ]:
# Remover features que não serão utilizadas
cols_drop = [
    'Customers', 
    'Open', 
    'PromoInterval', 
    'MonthMap'
]

df1.drop(
    columns=cols_drop, 
    inplace=True
)

In [ ]:
list(df1.columns)

## Salvar Dados Tratados

In [ ]:
pkl.dump(df1,open('../data/cleansed/df_cleansed.pkl','wb'))